# Session 3: Data Management in Structural Biology

## Learning Objectives

By the end of this session, you will understand:
1. **WHY** data management matters in structural biology
2. **HOW** to think about biological data as FLOWS between processors
3. **WHAT** makes protos different from "just loading files"
4. **HOW** computational analysis can be used to investigate biological hypotheses

## The Big Picture

In structural biology, we don't just "look at structures." We:
- Extract **COORDINATES** from PDB/mmCIF files
- Calculate **DISTANCES** between atoms
- Identify **INTERACTIONS** (H-bonds, hydrophobic, pi-stacking)
- Map positions using **NUMBERING SCHEMES** (like GRN)
- Connect to **EXPERIMENTAL DATA** (pEC50, Emax)

**Each step transforms data from one form to another. THIS IS DATA MANAGEMENT.**

---

**Paper:** Zhao et al. Nature 2023 - PCO371/PTH1R/Class B GPCR

In [1]:
# Setup paths and protos
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np

WORKSHOP_ROOT = Path.cwd()
PROTOS_SRC = WORKSHOP_ROOT / "protos" / "src"
sys.path.insert(0, str(PROTOS_SRC))

# CRITICAL: Set data path BEFORE importing any processors!
import protos
DATA_ROOT = WORKSHOP_ROOT / "materials" / "session3" / "data"
protos.set_data_path(str(DATA_ROOT))

print(f"Data root: {DATA_ROOT}")
print("""
    ^-- This single line is MORE IMPORTANT than you might think.
    
    It tells protos WHERE to store everything:
    - Downloaded structures -> data/structure/mmcif/
    - Extracted sequences  -> data/sequence/fasta/
    - Calculated properties -> data/property/tables/
    
    No more scattered files!
""")

RDKit not available. Some SDF functionality will be limited.
RDKit not available. Some conversion functions will be limited.
RDKit not available. Some ligand functionality will be limited.


Data root: C:\Users\hidbe\PycharmProjects\ProteinProtosWorkshop\materials\session3\data

    ^-- This single line is MORE IMPORTANT than you might think.

    It tells protos WHERE to store everything:
    - Downloaded structures -> data/structure/mmcif/
    - Extracted sequences  -> data/sequence/fasta/
    - Calculated properties -> data/property/tables/

    No more scattered files!



In [2]:
# Import processors
from protos.processing.structure import StructureProcessor
from protos.processing.sequence import SequenceProcessor
from protos.processing.grn import GRNProcessor
from protos.processing.property import PropertyProcessor
from protos.processing.structure.ligand_interactions import LigandInteractionAnalyzer

---

## Part 1: Framing the Biological Question

From Zhao et al. Nature 2023:

> "PCO371 is a small molecule that activates PTH1R (a GPCR) from INSIDE 
> the transmembrane bundle - completely different from how natural 
> hormones bind on the outside."

**PAPER CLAIMS:**
1. PCO371 binds in a pocket formed by TM2, TM3, TM6, TM7
2. Key residues: R219 (2x46), H223 (2x50), E302 (3x50), L413 (6x45), M414 (6x46), P415 (6x47), L416 (6x48), F417 (6x49)
3. P415 (6x47) is critical - mutating it to Ala abolishes activity

**OUR APPROACH:**
1. Load the structure (8JR9)
2. Find the ligand atoms
3. Calculate distances to nearby residues  
4. Compare our calculated contacts with the paper's claims

This is **DATA PROCESSING**: Structure file -> Coordinates -> Distance matrix -> Contact list

In [5]:
# Initialize StructureProcessor
struct_proc = StructureProcessor(name="pco371_analysis")
print(f"StructureProcessor initialized at: {struct_proc.path_cif_dir}")

# Define structures with biological context
structures = {
    '8jr9': {'description': 'PCO371-PTH1R-Gs (MAIN STRUCTURE)', 'purpose': 'Primary binding analysis'},
    '6nbf': {'description': 'LA-PTH-PTH1R-Gs (peptide reference)', 'purpose': 'Compare small molecule vs peptide'},
    '6x1a': {'description': 'PF-06882961-GLP1R-Gs (another small molecule)', 'purpose': 'Cross-receptor comparison'},
    '7f16': {'description': 'TIP39-PTH2R-Gs (related receptor)', 'purpose': 'Why doesnt PCO371 work on PTH2R?'}
}

# Load structures
print("Loading structures...")
loaded = {}
for pdb_id, info in structures.items():
    try:
        df = struct_proc.load_entity(pdb_id)
        if df is not None and len(df) > 0:
            loaded[pdb_id] = df
            chains = list(df['auth_chain_id'].unique())
            print(f"  [OK] {pdb_id}: {len(df):,} atoms, chains {chains}")
            print(f"       Purpose: {info['purpose']}")
        else:
            print(f"  [--] {pdb_id}: Will be downloaded on first use")
    except Exception as e:
        print(f"  [!!] {pdb_id}: {e}")

print(f"Loaded {len(loaded)} structure(s)")

2025-12-19 00:21:46,021 - StructureProcessor.pco371_analysis - INFO - Initialized StructureProcessor 'pco371_analysis' at C:\Users\hidbe\PycharmProjects\ProteinProtosWorkshop\materials\session3\data\structure


StructureProcessor initialized at: C:\Users\hidbe\PycharmProjects\ProteinProtosWorkshop\materials\session3\data\structure\mmcif
Loading structures...
  [OK] 8jr9: 7,924 atoms, chains ['A', 'B', 'G', 'N', 'R']
       Purpose: Primary binding analysis
  [OK] 6nbf: 9,424 atoms, chains ['R', 'P', 'A', 'B', 'G', 'N']
       Purpose: Compare small molecule vs peptide
  [--] 6x1a: Will be downloaded on first use
  [--] 7f16: Will be downloaded on first use
Loaded 2 structure(s)


---

## Part 2: Extracting the Ligand

**BIOLOGICAL CONCEPT:** The 8JR9 structure contains receptor, G-protein, AND ligand.
We need to FIND the ligand (PCO371) among ~8000 atoms.

**DATA MANAGEMENT CONCEPT:** This is **FILTERING** - selecting a subset based on criteria.



In [8]:
if '8jr9' in loaded:
    structure_8jr9 = loaded['8jr9']
    
    # Initialize interaction analyzer
    analyzer = LigandInteractionAnalyzer(structure_8jr9)
    
    # Extract ligands
    print("Extracting ligands from 8JR9...")
    ligands = analyzer.extract_ligands(exclude_common=True)
    
    print(f"Found {len(ligands)} ligand(s):")
    for lig in ligands:
        print(f"  - {lig['res_name3l']} (chain {lig['chain_id']}): {lig['num_atoms']} atoms")
        print(f"    Centroid: ({lig['centroid'][0]:.1f}, {lig['centroid'][1]:.1f}, {lig['centroid'][2]:.1f})")
    
    # Find main ligand (>20 atoms)
    pco371 = None
    for lig in ligands:
        if lig['num_atoms'] > 20:
            pco371 = lig
            break
    
    if pco371:
        print(f">>> FOUND MAIN LIGAND: {pco371['res_name3l']} with {pco371['num_atoms']} atoms")
        print(f"    Data reduction: 7,924 atoms -> {pco371['num_atoms']} atoms ({7924//pco371['num_atoms']}x)")
else:
    print("Structure 8jr9 not loaded")

Extracting ligands from 8JR9...
Found 1 ligand(s):
  - KHF (chain R): 75 atoms
    Centroid: (118.8, 107.3, 104.8)
>>> FOUND MAIN LIGAND: KHF with 75 atoms
    Data reduction: 7,924 atoms -> 75 atoms (105x)


---

## Part 3: Calculating Binding Site Residues (KEY CONCEPT!)

**BIOLOGICAL CONCEPT:** A "binding site" is the set of protein residues close enough to interact with the ligand. Typically within 4-5 Angstroms.

**DATA MANAGEMENT CONCEPT:** We compute a **DISTANCE MATRIX**:
- Ligand atoms (N atoms) x Protein atoms (M atoms)
- Result: N x M matrix of distances

**THE MATH:**


This transforms: **Coordinates (x,y,z) -> Distances (A) -> Contact list**

In [ ]:
if pco371 and '8jr9' in loaded:
    # GRN mapping for key PTH1R residues (Class B numbering)
    grn_mapping = {
        219: '2x46', 220: '2x47', 221: '2x48', 222: '2x49', 223: '2x50',
        224: '2x51', 225: '2x52', 226: '2x53',
        299: '3x47', 300: '3x48', 301: '3x49', 302: '3x50', 303: '3x51',
        306: '3x54',
        411: '6x43', 412: '6x44', 413: '6x45', 414: '6x46', 415: '6x47',
        416: '6x48', 417: '6x49',
        454: '7x52', 455: '7x53', 456: '7x54', 457: '7x55', 458: '7x56',
        459: '7x57', 460: '7x58',
        462: '8x47', 463: '8x48', 464: '8x49', 465: '8x50'
    }
    
    # Get binding site residues
    cutoff = 5.0  # Angstroms
    binding_residues = analyzer.get_binding_site_residues(pco371['atoms'], cutoff=cutoff)
    
    if not binding_residues.empty:
        print(f"BINDING SITE RESIDUES (calculated, within {cutoff} A of ligand):")
        print("-" * 80)
        print(f"{'Residue':<12} {'GRN':<10} {'Chain':<8} {'Distance (A)':<14} {'Contact Atoms':<25}")
        print("-" * 80)
        
        # Sort by distance
        binding_residues_sorted = binding_residues.sort_values('min_distance')
        
        # Paper key residues by GRN
        paper_grn = {'2x46', '2x50', '2x53', '3x47', '3x50', '6x43', '6x44', 
                     '6x45', '6x46', '6x47', '6x48', '6x49', '7x56', '7x57',
                     '8x47', '8x49'}
        
        for _, row in binding_residues_sorted.head(25).iterrows():
            res_label = f"{row['res_name']}{row['res_id']}"
            res_id = int(row['res_id']) if str(row['res_id']).isdigit() else 0
            grn = grn_mapping.get(res_id, '-')
            contact_str = ', '.join(row['contact_atoms'][:3])
            if len(row['contact_atoms']) > 3:
                contact_str += '...'
            marker = "*" if grn in paper_grn else " "
            print(f"{marker} {res_label:<10} {grn:<10} {row['chain_id']:<8} {row['min_distance']:<14.2f} {contact_str:<25}")
        
        print("-" * 80)
        print("* = Residue at GRN position mentioned in Zhao et al. 2023")
        print(f"Total binding site residues: {len(binding_residues)}")
        
        # Save to file
        binding_site_file = DATA_ROOT / "pco371_binding_site.csv"
        binding_residues_sorted.to_csv(binding_site_file, index=False)
        print()
        print(f"Saved to: {binding_site_file}")
else:
    print("Ligand not available for analysis")

---

## Part 4: Detecting Molecular Interactions

**BIOLOGICAL CONCEPT:** Proteins and ligands interact through specific forces:
- **Hydrogen bonds** (N-H...O, O-H...N): ~2.5-3.5 A
- **Hydrophobic contacts** (C...C): ~3.5-4.5 A  
- **Pi-stacking** (aromatic rings): ~4-5 A
- **Salt bridges** (charged groups): ~4 A

**DATA MANAGEMENT CONCEPT:** Each interaction type requires DIFFERENT calculations:
- H-bonds: distance + angle criteria
- Hydrophobic: distance + atom type filter
- Pi-stacking: ring centroid distance + normal angle

**THE POWER OF ABSTRACTION:** You ask "What are the interactions?" - the processor computes everything.

In [ ]:
if 'binding_residues' in dir() and not binding_residues.empty:
    # Categorize residues by type
    hydrophobic_res = ['ALA', 'VAL', 'LEU', 'ILE', 'MET', 'PHE', 'TRP', 'PRO']
    polar_res = ['SER', 'THR', 'ASN', 'GLN', 'TYR', 'CYS']
    charged_res = ['ARG', 'LYS', 'HIS', 'ASP', 'GLU']
    
    h_count = len(binding_residues[binding_residues['res_name'].isin(hydrophobic_res)])
    p_count = len(binding_residues[binding_residues['res_name'].isin(polar_res)])
    c_count = len(binding_residues[binding_residues['res_name'].isin(charged_res)])
    
    print("INTERACTION SUMMARY (calculated from structure):")
    print("=" * 60)
    print(f"  Total binding residues:     {len(binding_residues)}")
    print(f"  Hydrophobic residues:       {h_count}")
    print(f"  Polar residues:             {p_count}")
    print(f"  Charged residues:           {c_count}")
    
    # Aromatic residues for potential pi-stacking
    aromatic = binding_residues[binding_residues['res_name'].isin(['PHE', 'TYR', 'TRP', 'HIS'])]
    if not aromatic.empty:
        print("  Aromatic residues (potential pi-stacking):")
        for _, row in aromatic.iterrows():
            print(f"    {row['res_name']}{row['res_id']}: {row['min_distance']:.2f} A")
    
    # Report TM6 residues found (residues 411-417)
    print()
    print("=" * 60)
    print("TM6 residues found in binding site (residues 411-420):")
    found_tm6 = []
    for _, row in binding_residues.iterrows():
        res_id = int(row['res_id']) if str(row['res_id']).isdigit() else 0
        if 411 <= res_id <= 420:
            found_tm6.append(f"{row['res_name']}{row['res_id']}")
    if found_tm6:
        print(f"  Found: {', '.join(found_tm6)}")
    else:
        print("  None found")
    
    print()
    print("-" * 60)
    print("PAPER EXPECTATION (Zhao et al. 2023):")
    print("  Key binding residues (by GRN):")
    print("    2x46 (R219), 2x50 (H223), 2x53 (L226)")
    print("    3x47 (I299), 3x50 (E302)")
    print("    6x43 (L411), 6x44 (V412), 6x45 (L413), 6x46 (M414)")
    print("    6x47 (P415), 6x48 (L416), 6x49 (F417)")
    print("    7x56 (I458), 7x57 (Y459)")
    print("    8x47 (C462), 8x49 (G464)")
    print("-" * 60)

---

## Part 5: Generic Residue Numbering (GRN) - The Universal Language

**BIOLOGICAL CONCEPT:** Different GPCRs have different sequence lengths, but their transmembrane helices are structurally conserved.

GRN assigns **UNIVERSAL** position numbers:
-  = "position 47 in transmembrane helix 6, Class B numbering"

**DATA MANAGEMENT CONCEPT:** GRN is a MAPPING:


This enables **CROSS-RECEPTOR COMPARISON**:
- P415 in PTH1R = L370 in PTH2R = **same structural position**!

**WHY THIS MATTERS:**
- Without GRN: "Position 415 is important" (only true for PTH1R)
- With GRN: "Position 6.47b is important" (true for ALL Class B GPCRs)

In [ ]:
# Load GRN data from Session 1
grn_file = WORKSHOP_ROOT / "materials" / "session1" / "solution" / "nature_figure_gpcr_b_by_hand.csv"

try:
    grn_df = pd.read_csv(grn_file, skiprows=1)
    print("GRN BINDING POCKET (from Session 1):")
    print("-" * 60)
    
    # Show TM6 positions
    tm6_cols = [col for col in grn_df.columns if col.startswith('6')]
    if tm6_cols:
        print()
        print("TM6 positions (GRN 6x43-6x49):")
        for col in tm6_cols:
            values = grn_df[col].dropna().tolist()
            if values:
                unique = set(str(v) for v in values if str(v).isalpha())
                if len(unique) == 1:
                    print(f"  {col}: {list(unique)[0]} (conserved across receptors)")
                else:
                    print(f"  {col}: Variable ({', '.join(list(unique)[:3])})")
    
    print()
    print("PAPER OBSERVATION (Zhao et al. 2023):")
    print("-" * 45)
    print("- Position 6x47: P (PTH1R) vs L (PTH2R)")
    print("- Rescue experiment: PTH2R(L370P) gains PCO371 response")
    print()

except FileNotFoundError:
    print(f"Session 1 data not found at: {grn_file}")
    print("Run Session 1 first to extract the GRN table.")

---

## Part 6: Connecting Structure to Function

**BIOLOGICAL CONCEPT:** Structure determines function. Mutations in the binding site should affect ligand activity.

**DATA MANAGEMENT CONCEPT:** We're JOINING two datasets:
- **Structural data:** which residues contact ligand
- **Functional data:** which mutations affect activity (pEC50, Emax)

**JOIN KEY:** Residue identity (e.g., "P415A")

This is the **CORE OF DATA SCIENCE**: Combining datasets to reveal relationships invisible in either alone.

In [ ]:
# Load mutant activity data
mutant_file = DATA_ROOT / "pth1r_mutant_activity.csv"

try:
    mutant_df = pd.read_csv(mutant_file)
    print("MUTANT ACTIVITY DATA (from Zhao et al. 2023):")
    print("-" * 60)
    display(mutant_df)
    
    wt_pec50 = mutant_df[mutant_df['Mutant'] == 'WT']['pEC50'].iloc[0]
    
    print()
    print("=" * 60)
    print(f"WT pEC50 = {wt_pec50:.2f}")
    
    print()
    print("Mutations with enhanced activity (pEC50 > WT + 0.2):")
    enhancing = mutant_df[(mutant_df['pEC50'].notna()) & (mutant_df['pEC50'] > wt_pec50 + 0.2)]
    if enhancing.empty:
        print("  None found")
    else:
        for _, row in enhancing.sort_values('pEC50', ascending=False).iterrows():
            delta = row['pEC50'] - wt_pec50
            print(f"  {row['Mutant']:6} ({row['GRN']:>6}): pEC50 = {row['pEC50']:.2f} (+{delta:.2f})")
    
    print()
    print("Mutations with reduced activity:")
    reduced = mutant_df[(mutant_df['pEC50'].notna()) & (mutant_df['pEC50'] < wt_pec50 - 0.2) & (mutant_df['Mutant'] != 'WT')]
    for _, row in reduced.sort_values('pEC50').iterrows():
        delta = row['pEC50'] - wt_pec50
        print(f"  {row['Mutant']:6} ({row['GRN']:>6}): pEC50 = {row['pEC50']:.2f} ({delta:.2f})")
    
    print()
    print("Mutations with no detectable response:")
    abolishing = mutant_df[mutant_df['pEC50'].isna()]
    for _, row in abolishing.iterrows():
        print(f"  {row['Mutant']:6} ({row['GRN']:>6}): N/A")
    
except FileNotFoundError:
    print(f"Mutant data not found at: {mutant_file}")

---

## Part 7: Comparing Calculated Data with Paper Claims

This section presents our calculated findings alongside the published claims.
> The reader should evaluate whether the computational results support the paper's conclusions.

In [ ]:
print("COMPARISON: OUR CALCULATIONS vs PAPER CLAIMS")
print("=" * 70)

# GRN mapping for key PTH1R residues
grn_mapping = {
    219: '2x46', 223: '2x50', 226: '2x53',
    299: '3x47', 302: '3x50',
    411: '6x43', 412: '6x44', 413: '6x45', 414: '6x46', 415: '6x47',
    416: '6x48', 417: '6x49',
    458: '7x56', 459: '7x57',
    462: '8x47', 463: '8x48', 464: '8x49'
}

print()
print("--- PAPER CLAIM 1: PCO371 binds in TM bundle (TM2/TM3/TM6/TM7/H8) ---")
print("Paper residues (by GRN):")
print("  2x46, 2x50, 3x47, 3x50, 6x43, 6x44, 6x45, 6x46, 6x47, 6x48, 6x49, 7x56, 7x57, 8x47, 8x49")
print()
print("Our calculated contacts:")
if 'binding_residues' in dir() and not binding_residues.empty:
    # Group by approximate TM helix
    tm_ranges = {
        'TM2': (205, 235), 'TM3': (260, 310),
        'TM6': (400, 430), 'TM7': (445, 465), 'H8': (462, 475)
    }
    for tm, (start, end) in tm_ranges.items():
        tm_residues = []
        for _, row in binding_residues.iterrows():
            res_id = int(row['res_id']) if str(row['res_id']).isdigit() else 0
            if start <= res_id <= end:
                grn = grn_mapping.get(res_id, '')
                grn_str = f" ({grn})" if grn else ""
                tm_residues.append(f"{row['res_name']}{row['res_id']}{grn_str}")
        if tm_residues:
            print(f"  {tm}: {', '.join(tm_residues)}")

print()
print("--- PAPER CLAIM 2: Key residue mutations affect activity ---")
print("Paper mutation data (pEC50 values, delta from WT=6.74):")
print("  E302A (3x50): pEC50 = 5.85 (delta = -0.89)")
print("  L413A (6x45): pEC50 = 6.01 (delta = -0.73)")
print("  L226A (2x53): pEC50 = 6.07 (delta = -0.67)")
print("  I299A (3x47): pEC50 = 6.09 (delta = -0.65)")
print("  R219A (2x46): pEC50 = 6.23 (delta = -0.51)")
print("  H223A (2x50): pEC50 = 6.31 (delta = -0.43)")
print("  P415A (6x47): NO RESPONSE")
print()
print("Our calculated distances for these residues:")
if 'binding_residues' in dir() and not binding_residues.empty:
    key_res_ids = [302, 413, 226, 299, 219, 223, 415]
    for res_id in key_res_ids:
        match = binding_residues[binding_residues['res_id'] == res_id]
        grn = grn_mapping.get(res_id, '')
        if not match.empty:
            row = match.iloc[0]
            print(f"  {row['res_name']}{row['res_id']} ({grn}): {row['min_distance']:.2f} A from ligand")
        else:
            print(f"  Residue {res_id} ({grn}): not found in binding site (>{cutoff} A)")

print()
print("--- PAPER CLAIM 3: PTH2R lacks response due to L370 vs P415 ---")
print("Paper observation:")
print("  GRN 6x47: P (PTH1R) vs L (PTH2R)")
print("  Rescue experiment: PTH2R(L370P) gains PCO371 response")

print()
print("=" * 70)
print("NOTE: The above presents calculated data alongside paper claims.")
print("      Interpretation of agreement/disagreement is left to the reader.")

---

## Summary: The Data Management Perspective

Through **DATA MANAGEMENT**, we:

1. **LOADED** structures from PDB using standardized processors
2. **EXTRACTED** ligand atoms using filtering
3. **CALCULATED** distances using 3D coordinates  
4. **IDENTIFIED** interactions using geometric criteria
5. **MAPPED** positions using GRN numbering
6. **CONNECTED** structure to function using mutant data
7. **COMPARED** our calculations with published claims

This is the **WORKFLOW** of computational structural biology:

```
PDB file -> Structure DataFrame -> Ligand extraction -> Distance calculation -> Contact list -> GRN mapping -> Comparison with literature
```

Each step is **REPRODUCIBLE** because we used **MANAGED data flows**.

---

### Next Steps (Session 4):
Use **ProtOS-MCP** to orchestrate these workflows via natural language!

---

**See the data flow diagram:** 